In [3]:
import os
import shutil
import time
import numpy as np
import sys
from IPython.display import display, Markdown, HTML

In [3]:
# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..','src'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

# Developed components
from document_processor import DocumentProcessor
from vector_store import VectorStore
from retrieval import RetrievalSystem
from intent_classifier import IntentClassifier
from intent_handler import IntentHandlerManager, SessionState
from question_generator import QuestionGenerator
from answer_evaluator import AnswerEvaluator

In [4]:
# Create a clean test directory for ChromaDB
test_dir = "test_integration_data"
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
os.makedirs(test_dir, exist_ok=True)

### Initialize components
---

In [5]:
# Initialize the document processor
document_processor = DocumentProcessor(
    embedding_model="all-MiniLM-L6-v2",
    chunk_size=1000,  # Adjust chunk size as needed
    chunk_overlap=100
)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.48.1"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_rang

In [6]:
# Initialize ChromaDB vector store
vector_store = VectorStore(
    collection_name="test_collection",
    persist_directory=test_dir
)

Collection error: Collection test_collection does not exist.. Creating new collection.
Created new collection 'test_collection'


In [7]:
# Initialize retrieval system
retrieval_system = RetrievalSystem(
    vector_store=vector_store
)

In [8]:
question_generator = QuestionGenerator(
    retrieval_system=retrieval_system,
    use_local_llm=False,
    use_ollama=True
)

In [9]:
answer_evaluator = AnswerEvaluator(
    llm_backend='ollama',
    use_local_llm=False,
    use_ollama=True
)

In [10]:
classifier = IntentClassifier()

In [11]:
handler_manager = IntentHandlerManager(
    question_generator=question_generator,
    answer_evaluator=answer_evaluator,
    document_processor=document_processor
)

### Process Documents
---

In [12]:
def process_document(file_path):
    """Process a document and display results."""
    print(f"### Processing: {os.path.basename(file_path)}")
    
    start_time = time.time()
    chunks = document_processor.process_document(file_path)
    processing_time = time.time() - start_time
    
    print(f"**Extracted {len(chunks)} chunks in {processing_time:.2f} seconds**")
    
    # Display sample chunk info
    if chunks:
        display(Markdown("#### Sample chunk:"))
        display(Markdown(f"**Content (excerpt):**\n\n{chunks[0]['content'][:200]}..."))
        display(Markdown(f"**Embedding shape:** {chunks[0]['embedding'].shape}"))
        
        # Display metadata
        display(Markdown("**Metadata:**"))
        for key, value in chunks[0]['metadata'].items():
            display(Markdown(f"- {key}: {value}"))
    
    return chunks

In [13]:
document_paths = [
    "test_files/test.pdf",
    "test_files/test.pptx"
]

all_chunks = []

for doc_path in document_paths:
    if os.path.exists(doc_path):
        chunks = process_document(doc_path)
        if chunks:
            all_chunks.extend(chunks)
    else:
        display(Markdown(f"**Warning:** File not found: {doc_path}"))

print(f"Total chunks: {len(all_chunks)}")

### Processing: test.pdf


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


**Extracted 2128 chunks in 26.29 seconds**


#### Sample chunk:

**Content (excerpt):**

Creativity & Innovation
AHL3300 – AHL5300
By Cristian Zaelzer Ph.D.
7 to 9:50 PM – 200 Wilbrod 104
...

...

- source: test.pptx

- chunk_id: 0

- topics: ['graduation retention', 'prediction comparison', 'hydrant building', 'remote controls', '68 1146']

- page_number: 1

Total chunks: 8033


### Store in Vector DB
---

In [14]:
if all_chunks:
    display(Markdown(f"### Adding {len(all_chunks)} document chunks to ChromaDB"))
    
    start_time = time.time()
    vector_store.add_documents(all_chunks)
    add_time = time.time() - start_time
    
    collection_size = vector_store.get_collection_size()
    print(f"**Added all chunks in {add_time:.2f} seconds**")
    print(f"**ChromaDB collection size:** {collection_size} documents")
    
    # Get available topics
    topics = vector_store.get_topics()
    display(Markdown(f"**Available topics:** {', '.join(topics)}"))
else:
    display(Markdown("**No chunks extracted. Check your document paths.**"))

### Adding 8033 document chunks to ChromaDB

Added 8033 documents to ChromaDB collection
**Added all chunks in 2.62 seconds**
**ChromaDB collection size:** 8033 documents


**Available topics:** 68 1146, graduation retention, hydrant building, inclusion dei, innovation diversity, person assigned, prediction comparison, remote controls, root causes, zaelzer ph

### Test Search in Vector DB
---

In [15]:
test_queries = [
    "What is SCAMPER?",
    "What should leaders possess?",
    "What is Support Vector Machine?",
    "What is the project scope?"
]

# Function to display search results nicely
def display_search_results(query, results):
    display(Markdown(f"### Query: '{query}'"))
    
    if not results:
        display(Markdown("*No results found*"))
        return
        
    for i, result in enumerate(results):
        display(Markdown(f"#### Result {i+1} (Score: {result.get('similarity', 0):.4f})"))
        display(Markdown(f"**Source:** {result['metadata'].get('source', 'Unknown')}, " +
                          f"Page/Slide: {result['metadata'].get('page_number', 'Unknown')}"))
        display(Markdown(f"**Content:**\n\n{result['content'][:300]}..."))
        display(Markdown("---"))

In [16]:
# Run searches
for query in test_queries:
    start_time = time.time()
    results = retrieval_system.retrieve(query, top_k=3)
    search_time = time.time() - start_time
    
    display(Markdown(f"## Search Results ({search_time:.2f} seconds)"))
    display_search_results(query, results)

## Search Results (0.03 seconds)

### Query: 'What is SCAMPER?'

#### Result 1 (Score: 0.4750)

**Source:** test.pptx, Page/Slide: 5

**Content:**

xamples/

...

...

---

#### Result 3 (Score: 0.4754)

**Source:** test.pdf, Page/Slide: 17

**Content:**

e project.
2.
You can use a template here in, S.M.A.R.T. Goals Template. 
Activity
S.M.A.R.T. Goals
...

---

### Retrieval System
---

In [18]:
retrieval_system.retrieve(
                'characteristics of a leader', 
                100
            )

[{'id': 'doc_622_622',
  'content': ' a \ngood leader?\n\n[IMAGE 1 CAPTION: a woman standing in front of a group of people ]\n',
  'metadata': {'chunk_id': 622,
   'page_number': 7,
   'source': 'test.pdf',
   'topics': 'innovation diversity,inclusion dei,zaelzer ph,person assigned,root causes'},
  'score': 0.7570015788078308,
  'similarity': 0.5691514521452649,
  'adjusted_score': 0.5691514521452649},
 {'id': 'doc_623_623',
  'content': 'a \ngood leader?\n\n[IMAGE 1 CAPTION: a woman standing in front of a group of people ]\n',
  'metadata': {'chunk_id': 623,
   'page_number': 7,
   'source': 'test.pdf',
   'topics': 'innovation diversity,inclusion dei,zaelzer ph,person assigned,root causes'},
  'score': 0.7570015788078308,
  'similarity': 0.5691514521452649,
  'adjusted_score': 0.5691514521452649},
 {'id': 'doc_606_606',
  'content': 'Leaders should model \ncuriosity and openness.\nA good leader effectively \ncommunicates their vision, \nactively listens to their team, \nfosters stron

In [21]:
retrieval_system.retrieve_for_question_generation(
                ['characteristics of a leader','details about support vector machine'], 
                100
            )

[{'id': 'doc_5486_3358',
  'content': 'Support Vector Machine\nMethodology, Results and Analysis\n‹#›\n',
  'metadata': {'chunk_id': 3358,
   'page_number': 37,
   'source': 'test.pptx',
   'topics': 'graduation retention,prediction comparison,hydrant building,remote controls,68 1146'},
  'score': 0.7386980652809143,
  'similarity': 0.5751429877150258,
  'adjusted_score': 0.5751429877150258,
  'quality_score': 0.8627144815725387},
 {'id': 'doc_5574_3446',
  'content': ' of paper ]\nSupport Vector Machine\nMethodology - Technical Implementation\n',
  'metadata': {'chunk_id': 3446,
   'page_number': 38,
   'source': 'test.pptx',
   'topics': 'graduation retention,prediction comparison,hydrant building,remote controls,68 1146'},
  'score': 0.774863064289093,
  'similarity': 0.5634237480740757,
  'adjusted_score': 0.5634237480740757,
  'quality_score': 0.8451356221111135},
 {'id': 'doc_5569_3441',
  'content': 'piece of paper ]\nSupport Vector Machine\nMethodology - Technical Implementatio

## Question Generation
---

#### MC

In [22]:
# Test the question generation
easy_mc_question = question_generator.generate_question(['characteristics of a leader','diversity and inclusion'],
                                                   question_type="multiple-choice", 
                                                     difficulty="easy")

# Display the generated question
print("Generated Question:", easy_mc_question.get("question"))
print("Options:", easy_mc_question.get("options"))
print("Answer:", easy_mc_question.get("answer"))
print("Explanation:", easy_mc_question.get("explanation"))

Generated Question: What characteristic does a good leader demonstrate by articulating goals and ideas clearly?
Options: ['Empowerment: Provides autonomy and trust to team members to make decisions and own their work.', 'Clear Communication: Articulates goals and ideas clearly, ensuring everyone understands the direction.', "Active Listening: Pays attention to team members' concerns and feedback.", 'Accountability: Takes responsibility for both successes and failures.']
Answer: B
Explanation: A good leader demonstrates clear communication by articulating their goals and ideas clearly, which is essential for ensuring everyone understands the direction. Option A is incorrect because empowerment is a separate characteristic that involves providing autonomy and trust to team members. Option C is also incorrect because active listening is about paying attention to team members' concerns and feedback, not articulating goals and ideas. Option D is incorrect because accountability is about tak

In [23]:
medium_mc_question = question_generator.generate_question(['characteristics of a leader','diversity and inclusion'],
                                                   question_type="multiple-choice", 
                                                     difficulty="medium")

# Display the generated question
print("Generated Question:", medium_mc_question.get("question"))
print("Options:", medium_mc_question.get("options"))
print("Answer:", medium_mc_question.get("answer"))
print("Explanation:", medium_mc_question.get("explanation"))

Generated Question: Which characteristic of a good leader does NOT contribute to effective communication?
Options: ['Articulates goals and ideas clearly, ensuring everyone understands the direction.', 'Provides autonomy and trust to team members to make decisions and own their work.', "Pays attention to team members' concerns and feedback.", 'Acts ethically and consistently with their values, setting a moral example.']
Answer: B
Explanation: Providing autonomy and trust (Option B) is an aspect of empowerment, not communication. Effective communication involves articulating goals clearly (Option A), actively listening to team members' concerns (Option C), and maintaining integrity (Option D).


In [24]:
hard_mc_question = question_generator.generate_question(['characteristics of a leader','diversity and inclusion'],
                                                   question_type="multiple-choice", 
                                                     difficulty="hard")

# Display the generated question
print("Generated Question:", hard_mc_question.get("question"))
print("Options:", hard_mc_question.get("options"))
print("Answer:", hard_mc_question.get("answer"))
print("Explanation:", hard_mc_question.get("explanation"))

Generated Question: In a team where members have diverse backgrounds and experiences, which leadership approach would be most effective in promoting collaboration and innovation?
Options: ['A. Empowering individuals to make decisions based on their own expertise and judgment.', "B. Encouraging open communication by actively listening to each member's concerns and ideas.", 'C. Fostering a competitive environment where team members are pitted against each other to achieve individual goals.', 'D. Assigning tasks based solely on seniority, regardless of individual strengths or interests.']
Answer: A
Explanation: Empowering individuals allows for autonomy and trust, which can lead to increased motivation and creativity in a diverse team. This approach also acknowledges the unique experiences and perspectives each member brings to the table. Options B, C, and D are incorrect because while they may promote some level of collaboration or innovation, they do not directly address the needs of a 

#### Free-text

In [28]:
# Test the question generation
easy_free_text_question = question_generator.generate_question(['characteristics of a leader','diversity and inclusion','psychological safety'],
                                                    question_type="free-text", 
                                                     difficulty="easy")

# Display the generated question
print("Generated Question:", easy_free_text_question.get("question"))
print("Options:", easy_free_text_question.get("options"))
print("Answer:", easy_free_text_question.get("answer"))
print("Key Points:", easy_free_text_question.get("key_points"))

Question validation failed on attempt 1: Free-text question needs at least 2 key points


Generated Question: Explain how a diverse team can foster innovation and describe an example of how equity, diversity, and inclusion (EDI) principles can be applied in a real-world scenario to address a specific challenge.
Options: None
Answer: A diverse team can foster innovation by bringing together individuals with different backgrounds, experiences, and expertise. This diversity enables the team to approach problems from multiple angles, leading to more creative solutions. For example, consider a company developing a new product for an aging population. A team consisting of members from different age groups, ethnicities, and abilities would be better equipped to understand the needs and challenges faced by this demographic. By applying EDI principles, the company can ensure that the product is not only inclusive but also accessible to people with disabilities. This might involve consulting with accessibility experts, conducting user testing with individuals from diverse backgrounds

In [29]:
# Test the question generation
medium_free_text_question = question_generator.generate_question(['characteristics of a leader','diversity and inclusion','psychological safety'],
                                                    question_type="free-text", 
                                                     difficulty="medium")

# Display the generated question
print("Generated Question:", medium_free_text_question.get("question"))
print("Options:", medium_free_text_question.get("options"))
print("Answer:", medium_free_text_question.get("answer"))
print("Key Points:", medium_free_text_question.get("key_points"))

Generated Question: An organization has implemented various diversity and inclusion initiatives, but despite these efforts, a team of engineers from different backgrounds is struggling to work together effectively. Analyze the potential reasons for this challenge, considering both the benefits and limitations of diverse teams as described in the context. Suggest strategies to improve collaboration and foster an inclusive environment within the team.
Options: None
Answer: The potential reasons for the challenge faced by the team of engineers from different backgrounds can be attributed to several factors. Firstly, the concept of diversity is often confused with inclusion, leading to a lack of understanding about how to work collaboratively in a diverse setting. Additionally, the idea that diverse teams offer a broader range of perspectives may lead to an expectation that every member will contribute equally, which can create pressure and anxiety among team members.To address these chall

In [30]:
# Test the question generation
hard_free_text_question = question_generator.generate_question(['characteristics of a leader','diversity and inclusion','psychological safety'],
                                                    question_type="free-text", 
                                                     difficulty="hard")

# Display the generated question
print("Generated Question:", hard_free_text_question.get("question"))
print("Options:", hard_free_text_question.get("options"))
print("Answer:", hard_free_text_question.get("answer"))
print("Key Points:", hard_free_text_question.get("key_points"))

Question validation failed on attempt 1: Free-text question needs at least 2 key points


Generated Question: While fostering psychological safety is crucial for innovation, some teams may struggle to balance this approach with the need for constructive feedback and accountability. What are potential risks and trade-offs associated with promoting open communication and idea sharing in a team, and how can leaders mitigate these risks while still encouraging creativity and innovation?
Options: None
Answer: Potential risks and trade-offs associated with promoting open communication and idea sharing include:* Overemphasis on avoiding criticism or conflict may lead to lack of accountability and ineffective decision-making.* Fear of ridicule or punishment may prevent team members from speaking up, leading to missed opportunities for growth and improvement.* Insufficient feedback mechanisms can result in unproductive ideas being pursued, wasting resources and time.To mitigate these risks, leaders can implement a balanced approach by:* Encouraging open communication while also esta

#### Knowledge Check

In [31]:
question_generator.generate_knowledge_check('test.pdf',5)

[{'question': 'What is the primary goal of Equity in the context of EDI?',
  'options': ['To provide equal opportunities for all individuals without considering their differences.',
   'To recognize and appreciate the unique characteristics and experiences of each individual.',
   'To create a fair environment where everyone has access to resources, opportunities, or benefits',
   'To actively promote diversity in the workplace'],
  'answer': 'C',
  'explanation': 'Equity is about creating a fair environment where everyone has equal access to resources, opportunities, or benefits. Option A is incorrect because it implies that equality is the same as equity, which is not necessarily true. Option B describes Diversity, not Equity. Option D is incorrect because promoting diversity is more related to Inclusion than Equity.',
  'type': 'multiple-choice',
  'metadata': {'difficulty': 'easy', 'topic': 'inclusion dei'},
  'document_id': 'test.pdf',
  'topic': 'inclusion dei',
  'difficulty': '

## Answer Evaluation
---

#### Multiple Choice

In [43]:
easy_mc_question

{'question': 'What characteristic does a good leader demonstrate by articulating goals and ideas clearly?',
 'options': ['Empowerment: Provides autonomy and trust to team members to make decisions and own their work.',
  'Clear Communication: Articulates goals and ideas clearly, ensuring everyone understands the direction.',
  "Active Listening: Pays attention to team members' concerns and feedback.",
  'Accountability: Takes responsibility for both successes and failures.'],
 'answer': 'B',
 'explanation': "A good leader demonstrates clear communication by articulating their goals and ideas clearly, which is essential for ensuring everyone understands the direction. Option A is incorrect because empowerment is a separate characteristic that involves providing autonomy and trust to team members. Option C is also incorrect because active listening is about paying attention to team members' concerns and feedback, not articulating goals and ideas. Option D is incorrect because accountabil

In [32]:
mc_correct_evaluation = answer_evaluator.evaluate_answer(easy_mc_question, 'B')
print(mc_correct_evaluation['is_correct'])
print(mc_correct_evaluation['feedback'])

True
Correct! Clear Communication: Articulates goals and ideas clearly, ensuring everyone understands the direction. is the right answer.


In [45]:
mc_correct_evaluation = answer_evaluator.evaluate_answer(easy_mc_question, 'Clear Communication')
print(mc_correct_evaluation['is_correct'])
print(mc_correct_evaluation['feedback'])

False
That's not quite right. Explanation: Clear Communication: Articulates goals and ideas clearly, ensuring everyone understands the direction.. A good leader demonstrates clear communication by articulating their goals and ideas clearly, which is essential for ensuring everyone understands the direction. Option A is incorrect because empowerment is a separate characteristic that involves providing autonomy and trust to team members. Option C is also incorrect because active listening is about paying attention to team members' concerns and feedback, not articulating goals and ideas. Option D is incorrect because accountability is about taking responsibility for both successes and failures, not communicating clearly.


In [42]:
mc_incorrect_evaluation = answer_evaluator.evaluate_answer(easy_mc_question, 'A')
print(mc_incorrect_evaluation['is_correct'])
print(mc_incorrect_evaluation['feedback'])

False
That's not quite right. Explanation: Clear Communication: Articulates goals and ideas clearly, ensuring everyone understands the direction.. A good leader demonstrates clear communication by articulating their goals and ideas clearly, which is essential for ensuring everyone understands the direction. Option A is incorrect because empowerment is a separate characteristic that involves providing autonomy and trust to team members. Option C is also incorrect because active listening is about paying attention to team members' concerns and feedback, not articulating goals and ideas. Option D is incorrect because accountability is about taking responsibility for both successes and failures, not communicating clearly.


#### Free-text

In [46]:
answer = 'A diverse team can foster innovation by bringing together individuals with different backgrounds, experiences, and expertise. This diversity enables the team to approach problems from multiple angles, leading to more creative solutions. For example, consider a company developing a new product for an aging population. A team consisting of members from different age groups, ethnicities, and abilities would be better equipped to understand the needs and challenges faced by this demographic. By applying EDI principles, the company can ensure that the product is not only inclusive but also accessible to people with disabilities. This might involve consulting with accessibility experts, conducting user testing with individuals from diverse backgrounds, and incorporating feedback to refine the design. Through this process, the team would be able to create a product that meets the needs of its target audience, ultimately driving business success while promoting social responsibility.'

In [47]:
ft_correct_evaluation = answer_evaluator.evaluate_answer(easy_free_text_question, answer)
print(ft_correct_evaluation['is_correct'])
print(ft_correct_evaluation['feedback'])

True
Excellent job on explaining how a diverse team can foster innovation! Your example of developing a product for an aging population is clear and well-supported. You've also done a great job applying EDI principles to ensure that the product is inclusive and accessible. One minor suggestion I have is to consider providing more specific examples or details about how consulting with accessibility experts and conducting user testing can be implemented in practice. However, overall, your answer demonstrates a strong understanding of the topic and effectively addresses all key points.


In [48]:
ft_partially_correct_evaluation = answer_evaluator.evaluate_answer(easy_free_text_question, 'To balance the need for diversity and inclusion with the potential risks of groupthink, organizations can employ several strategies. Firstly, they can encourage cross-functional collaboration by bringing together team members from different backgrounds, disciplines, and departments to work on projects. This helps to integrate diverse skill sets and viewpoints, generating innovative solutions that a single discipline might overlook.Secondly, organizations can foster an inclusive environment where everyone feels valued and respected. This involves actively creating opportunities for underrepresented groups to contribute their perspectives and ideas, and promoting empathy and understanding among team members.')
print(ft_partially_correct_evaluation['is_correct'])
print(ft_partially_correct_evaluation['feedback'])

False
Here is the evaluation of the student's answer:

{
    "is_correct": Partially,
    "feedback": "Hi Student, I can see that you've made a good effort to address the question. However, there are some areas where your answer deviates from the key points and reference answer.

Firstly, while you mention the importance of diversity in fostering innovation, you don't explicitly explain how EDI principles ensure that products or services are inclusive and accessible. This is an important aspect that needs to be addressed.

Secondly, your real-world scenario example is not as specific and detailed as the one provided in the reference answer. You could have chosen a more concrete example to illustrate how EDI principles can be applied in practice.

Lastly, while you mention cross-functional collaboration and fostering an inclusive environment, these points are not directly related to the key points I mentioned earlier. It's great that you're thinking about ways to promote diversity and i

In [49]:
ft_incorrect_evaluation = answer_evaluator.evaluate_answer(easy_free_text_question, "I don't know the answer lol")
print(ft_incorrect_evaluation['is_correct'])
print(ft_incorrect_evaluation['feedback'])

True
{
    "is_correct": false,
    "feedback": "Hi there! I can see that you're feeling uncertain about this question. That's completely okay! However, the correct answer requires a bit more thought and explanation. Let me break it down for you.

A diverse team can indeed foster innovation by bringing together individuals with different backgrounds, experiences, and expertise. This diversity enables the team to approach problems from multiple angles, leading to more creative solutions.

To address a specific challenge using EDI principles, let's consider an example: developing a product for an aging population. A team consisting of members from different age groups, ethnicities, and abilities would be better equipped to understand the needs and challenges faced by this demographic.

By applying EDI principles, the company can ensure that the product is not only inclusive but also accessible to people with disabilities. This might involve consulting with accessibility experts, conducti

## Sample Conversation (includes Intent Classification and Handling)
---

In [ ]:
def process_user_input(user_input, document_data=None):
    """Process user input through the classifier and handler."""
    
    # Classify the intent
    intent_data = classifier.classify(user_input)
    intent_type = intent_data["intent"]
    
    print(f"Classified Intent: {intent_type}")
    print(f"Intent Data: {intent_data}\n")
    
    # Add document data if provided
    if document_data:
        intent_data["document_data"] = document_data
    
    # Handle the intent
    response = handler_manager.handle_intent(intent_type, intent_data)
    
    print(f"Response: {response}\n")
    print("-" * 80)
    
    return response


def run_conversation_test(conversation):
    """Run a simulated conversation through the system."""
    print("=== CONVERSATION TEST ===\n")
    
    for i, user_input in enumerate(conversation):
        print(f"User: {user_input}")
        response = process_user_input(user_input)
        print(f"System: {response.get('text', 'No response text')}")
        print()

In [ ]:
# Define a conversation flow
conversation = [
    "Start review",
    "Hi, I want to upload a PDF document for review",
    "Let's use free-text questions instead of multiple choice",
    "I'd like to focus on Python programming topics",
    "Make it challenging",
    "Let's just do 3 questions",
    "Start the review session",
    "Object-oriented programming is a paradigm that uses objects to model real world problems",
    "How am I doing so far?",
    "Let's end the review",
    "Thanks for the help!"
]

run_conversation_test(conversation)